In [1]:
import pandas as pd

In [2]:
#Leemos la data
url=r'C:\Users\andre\Downloads\ObesityDataSet_raw_and_data_sinthetic.csv'
df=pd.read_csv(url)

In [3]:
#Hacemos que se vean rodas las columnas
pd.set_option('display.max_columns', None)

FAVC:" Do you eat high caloric food frequently? "

FCVC:" Do you usually eat vegetables in your meals? "

NCP:" How many main meals do you have daily? "

CAEC:" Do you eat any food between meals? "

SMOKE:" Do you smoke? "

CH2O:" How much water do you drink daily? "

SCC:" Do you monitor the calories you eat daily? "

FAF:" How often do you have physical activity? "

TUE:" How much time do you use technological devices such as cell phone, videogames, television, computer and others? "

CALC :" How often do you drink alcohol? "

MTRANS:" Which transportation do you usually use?

In [4]:
#Miramos tanto sus dtypes como si tiene nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             2111 non-null   float64
 1   Gender                          2111 non-null   object 
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   CALC                            2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   SCC                             2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  family_history_with_overweight  2111 non-null   object 
 12  FAF                             21

In [5]:
#Miramos si tiene datos nulos
df.isnull().sum()

Age                               0
Gender                            0
Height                            0
Weight                            0
CALC                              0
FAVC                              0
FCVC                              0
NCP                               0
SCC                               0
SMOKE                             0
CH2O                              0
family_history_with_overweight    0
FAF                               0
TUE                               0
CAEC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

### **Limpieza**

In [6]:
#Quitamos las mayusculas 
df.columns=df.columns.str.lower()

In [7]:
#Cambiamos los nombres de dos columnas "family_history_with_overweight" y "nobeyesdad"
df=df.rename(columns={"family_history_with_overweight": "family_history", "nobeyesdad": "obesity_level"})

In [9]:
#Calculo el índice de masa corporal (IMC) por nivel de obesidad
df['bmi'] = df['weight'] / (df['height'] / 100) ** 2

**Preparamos los datos**

In [10]:
#seleccionamos las columnas categoricas y numericas y las separamos en dataframes distintos.
potential_categorical_from_numerical = df.select_dtypes("number").loc[:, df.select_dtypes("number").nunique() < 20]

df_categorical = pd.concat([df.select_dtypes("object"), potential_categorical_from_numerical], axis=1)

df_numerical = df.select_dtypes("number").drop(columns=potential_categorical_from_numerical.columns)

In [11]:
# Realizar One-Hot Encoding usando pd.get_dummies()
df_encoded = pd.get_dummies(df_categorical, columns=['calc', 'caec', 'mtrans'], drop_first=True)

In [12]:
# Convertir 'gender' a formato binario
df_encoded['gender'] = df_encoded['gender'].apply(lambda x: 1 if x == 'Male' else 0)

# Asegurar que las columnas binarias estén en formato correcto (0 y 1)
binary_columns = ['favc', 'scc', 'smoke', 'family_history']
for col in binary_columns:
    df_encoded[col] = df_encoded[col].apply(lambda x: 1 if x == 'yes' else 0)

In [13]:
# Unimos los dos dataframes
df_final = pd.concat([df_encoded, df_numerical], axis=1)

In [14]:
df_final

,gender,favc,scc,smoke,family_history,obesity_level,calc_Frequently,calc_Sometimes,calc_no,caec_Frequently,caec_Sometimes,caec_no,mtrans_Bike,mtrans_Motorbike,mtrans_Public_Transportation,mtrans_Walking,age,height,weight,fcvc,ncp,ch2o,faf,tue,bmi
0,0,0,0,0,1,Normal_Weight,False,False,True,False,True,False,False,False,True,False,21.000000,1.620000,64.000000,2.0,3.0,2.000000,0.000000,1.000000,243865.264441
1,0,0,1,1,1,Normal_Weight,False,True,False,False,True,False,False,False,True,False,21.000000,1.520000,56.000000,3.0,3.0,3.000000,3.000000,0.000000,242382.271468
2,1,0,0,0,1,Normal_Weight,True,False,False,False,True,False,False,False,True,False,23.000000,1.800000,77.000000,2.0,3.0,2.000000,2.000000,1.000000,237654.320988
3,1,0,0,0,0,Overweight_Level_I,True,False,False,False,True,False,False,False,False,True,27.000000,1.800000,87.000000,3.0,3.0,2.000000,2.000000,0.000000,268518.518519
4,1,0,0,0,0,Overweight_Level_II,False,True,False,False,True,False,False,False,True,False,22.000000,1.780000,89.800000,2.0,1.0,2.000000,0.000000,0.000000,283423.810125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,1,0,0,1,Obesity_Type_III,False,True,False,False,True,False,False,False,True,False,20.976842,1.710730,131.408528,3.0,3.0,1.728139,1.676269,0.906247,449014.750007
2107,0,1,0,0,1,Obesity_Type_III,False,True,False,False,True,False,False,False,True,False,21.982942,1.748584,133.742943,3.0,3.0,2.005130,1.341390,0.599270,437419.233525
2108,0,1,0,0,1,Obesity_Type_III,False,True,False,False,True,False,False,False,True,False,22.524036,1.752206,133.689352,3.0,3.0,2.054193,1.414209,0.646288,435438.165841
2109,0,1,0,0,1,Obesity_Type_III,False,True,False,False,True,False,False,False,True,False,24.361936,1.739450,133.346641,3.0,3.0,2.852339,1.139107,0.586035,440715.354033


In [ ]:
# Eliminamos la columna 'obesity_level' de los features y lo colocamos como target
features = df_final.drop(columns = "obesity_level")
target = df_final[["obesity_level"]]